In [1]:
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

import optuna
import optuna.visualization as vis

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [4]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

#### **Parameters selection**

In [7]:
params = {
    'hidden_layer_sizes': [32, 64, 128],
    'alpha': [0.0001, 0.0],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid']
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

def objective(trial):
    model = keras.Sequential([
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(units=1, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            trial.suggest_categorical(
                'alpha',
                params['alpha']
            )
        ),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return accuracy

In [8]:
n_trials = 15

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-03-26 22:00:56,438] A new study created in memory with name: no-name-dab26778-8825-45a7-bfb2-b5224c5b255e


Epoch 1/5000
2280/2280 [==============================] - 8s 3ms/step - loss: 0.6049 - accuracy: 0.6554 - val_loss: 0.5989 - val_accuracy: 0.6557
Epoch 2/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5952 - accuracy: 0.6554 - val_loss: 0.5949 - val_accuracy: 0.6557
Epoch 3/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5928 - accuracy: 0.6554 - val_loss: 0.5985 - val_accuracy: 0.6557
Epoch 4/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5920 - accuracy: 0.6554 - val_loss: 0.5965 - val_accuracy: 0.6557
Epoch 5/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5901 - accuracy: 0.6554 - val_loss: 0.5945 - val_accuracy: 0.6557
Epoch 6/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5897 - accuracy: 0.6554 - val_loss: 0.5933 - val_accuracy: 0.6557
Epoch 7/5000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5888 - accuracy: 0.6554 - val_loss: 0.5929 -

[I 2023-03-26 22:02:49,866] Trial 0 finished with value: 0.5928604602813721 and parameters: {'hidden_layer_sizes': 128, 'activation': 'sigmoid', 'alpha': 0.01, 'batch_size': 128, 'max_iter': 5000}. Best is trial 0 with value: 0.5928604602813721.


Epoch 1/2000
9119/9119 [==============================] - 20s 2ms/step - loss: 0.7215 - accuracy: 0.6554 - val_loss: 0.7015 - val_accuracy: 0.6557
Epoch 2/2000
9119/9119 [==============================] - 15s 2ms/step - loss: 0.7211 - accuracy: 0.6554 - val_loss: 0.8502 - val_accuracy: 0.6557
Epoch 3/2000
9119/9119 [==============================] - 15s 2ms/step - loss: 0.7193 - accuracy: 0.6554 - val_loss: 0.7818 - val_accuracy: 0.6557
Epoch 4/2000
9119/9119 [==============================] - 16s 2ms/step - loss: 0.7251 - accuracy: 0.6554 - val_loss: 0.9150 - val_accuracy: 0.6557
Epoch 5/2000
9119/9119 [==============================] - 17s 2ms/step - loss: 0.7216 - accuracy: 0.6554 - val_loss: 0.7228 - val_accuracy: 0.6557
Epoch 6/2000
9119/9119 [==============================] - 19s 2ms/step - loss: 0.7215 - accuracy: 0.6554 - val_loss: 1.7849 - val_accuracy: 0.6557
Epoch 7/2000
9119/9119 [==============================] - 18s 2ms/step - loss: 0.7260 - accuracy: 0.6554 - val_loss: 0

[I 2023-03-26 22:08:03,837] Trial 1 finished with value: 0.6444234848022461 and parameters: {'hidden_layer_sizes': 32, 'activation': 'sigmoid', 'alpha': 0.46, 'batch_size': 32, 'max_iter': 2000}. Best is trial 0 with value: 0.5928604602813721.


...

[I 2023-03-26 22:42:44,488] Trial 12 finished with value: 0.5916972756385803 and parameters: {'hidden_layer_sizes': 64, 'activation': 'relu', 'alpha': 0.01, 'batch_size': 128, 'max_iter': 10000}. Best is trial 11 with value: 0.5911120772361755.


Epoch 1/10000
2280/2280 [==============================] - 11s 4ms/step - loss: 0.6018 - accuracy: 0.6554 - val_loss: 0.5963 - val_accuracy: 0.6557
Epoch 2/10000
2280/2280 [==============================] - 8s 4ms/step - loss: 0.5943 - accuracy: 0.6554 - val_loss: 0.5940 - val_accuracy: 0.6557
Epoch 3/10000
2280/2280 [==============================] - 8s 4ms/step - loss: 0.5925 - accuracy: 0.6554 - val_loss: 0.5938 - val_accuracy: 0.6557
Epoch 4/10000
2280/2280 [==============================] - 8s 4ms/step - loss: 0.5915 - accuracy: 0.6554 - val_loss: 0.5940 - val_accuracy: 0.6557
Epoch 5/10000
2280/2280 [==============================] - 8s 4ms/step - loss: 0.5907 - accuracy: 0.6554 - val_loss: 0.5916 - val_accuracy: 0.6557
Epoch 6/10000
2280/2280 [==============================] - 8s 3ms/step - loss: 0.5904 - accuracy: 0.6554 - val_loss: 0.5917 - val_accuracy: 0.6557
Epoch 7/10000
2280/2280 [==============================] - 9s 4ms/step - loss: 0.5897 - accuracy: 0.6554 - val_loss: 

[I 2023-03-26 22:46:33,061] Trial 13 finished with value: 0.5903142094612122 and parameters: {'hidden_layer_sizes': 64, 'activation': 'relu', 'alpha': 0.01, 'batch_size': 128, 'max_iter': 10000}. Best is trial 13 with value: 0.5903142094612122.


Epoch 1/10000
2280/2280 [==============================] - 7s 3ms/step - loss: 0.6023 - accuracy: 0.6554 - val_loss: 0.5985 - val_accuracy: 0.6557
Epoch 2/10000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5943 - accuracy: 0.6554 - val_loss: 0.5982 - val_accuracy: 0.6557
Epoch 3/10000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5926 - accuracy: 0.6554 - val_loss: 0.5951 - val_accuracy: 0.6557
Epoch 4/10000
2280/2280 [==============================] - 6s 3ms/step - loss: 0.5913 - accuracy: 0.6554 - val_loss: 0.5965 - val_accuracy: 0.6557
Epoch 5/10000
2280/2280 [==============================] - 9s 4ms/step - loss: 0.5907 - accuracy: 0.6554 - val_loss: 0.5938 - val_accuracy: 0.6557
Epoch 6/10000
2280/2280 [==============================] - 11s 5ms/step - loss: 0.5901 - accuracy: 0.6554 - val_loss: 0.5944 - val_accuracy: 0.6557
Epoch 7/10000
2280/2280 [==============================] - 19s 8ms/step - loss: 0.5896 - accuracy: 0.6554 - val_loss:

[I 2023-03-26 22:52:51,300] Trial 14 finished with value: 0.5910511612892151 and parameters: {'hidden_layer_sizes': 64, 'activation': 'relu', 'alpha': 0.01, 'batch_size': 128, 'max_iter': 10000}. Best is trial 13 with value: 0.5903142094612122.


In [9]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

Best hyperparameters:  {'hidden_layer_sizes': 64, 'activation': 'relu', 'alpha': 0.01, 'batch_size': 128, 'max_iter': 10000}
Best objective value:  0.5903142094612122


#### **Saving study**

In [10]:
import joblib

In [11]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']